In [83]:
base_url = "https://beacon-argo.eoscfe.mesocentre.uca.fr"

In [84]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [85]:
responseinfo = requests.get(
    f"{base_url}/api/query/available-columns",
)
params = responseinfo.json()

In [86]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("temp")  # Enter your search term here

Matching columns:
PROFILE_TEMP_QC
PROFILE_TEMP_QC.long_name
PROFILE_TEMP_QC.conventions
PROFILE_TEMP_QC._FillValue
TEMP
TEMP.long_name
TEMP.standard_name
TEMP._FillValue
TEMP.units
TEMP.valid_min
TEMP.valid_max
TEMP.C_format
TEMP.FORTRAN_format
TEMP.resolution
TEMP_QC
TEMP_QC.long_name
TEMP_QC.conventions
TEMP_QC._FillValue
TEMP_ADJUSTED
TEMP_ADJUSTED.long_name
TEMP_ADJUSTED.standard_name
TEMP_ADJUSTED._FillValue
TEMP_ADJUSTED.units
TEMP_ADJUSTED.valid_min
TEMP_ADJUSTED.valid_max
TEMP_ADJUSTED.C_format
TEMP_ADJUSTED.FORTRAN_format
TEMP_ADJUSTED.resolution
TEMP_ADJUSTED_QC
TEMP_ADJUSTED_QC.long_name
TEMP_ADJUSTED_QC.conventions
TEMP_ADJUSTED_QC._FillValue
TEMP_ADJUSTED_ERROR
TEMP_ADJUSTED_ERROR.long_name
TEMP_ADJUSTED_ERROR._FillValue
TEMP_ADJUSTED_ERROR.units
TEMP_ADJUSTED_ERROR.C_format
TEMP_ADJUSTED_ERROR.FORTRAN_format
TEMP_ADJUSTED_ERROR.resolution


In [87]:
mindate = "2018-01-01"  # yyyy-mm-dd
maxdate = "2019-01-01"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
mindepth = 0
maxdepth = 10

In [88]:
def query(mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth):
    body = {
        "query_parameters": [
            {
                "column_name": "TEMP",
            },
            {
                "column_name": "TEMP_QC",
            },
            {"column_name": "TEMP.units", "alias": "TEMP_units"},
            {"column_name": "TEMP.standard_name", "alias": "TEMP_standard_name"},
            {"column_name": "PSAL", "alias": "PSAL"},
            {"column_name": "PSAL.standard_name", "alias": "PSAL_standard_name"},
            {"column_name": "JULD", "alias": "TIME"},
            {"column_name": "PRES", "alias": "DEPTH"},
            {"column_name": "LONGITUDE", "alias": "LONGITUDE"},
            {"column_name": "LATITUDE", "alias": "LATITUDE"},
            {
                # Special column name to get the dataset name from Beacon Binary Format
                "column_name": "@identifier",
                "alias": "DATASET",
            },
        ],
        "filters": [
            {
                "for_query_parameter": "TIME",
                "min": mindate,
                "max": maxdate,
            },
            {"for_query_parameter": "DEPTH", "min": mindepth, "max": maxdepth},
            {"for_query_parameter": "LONGITUDE", "min": minlon, "max": maxlon},
            {"for_query_parameter": "LATITUDE", "min": minlat, "max": maxlat},
        ],
        "output": {"format": "netcdf"},
    }
    return body


query_body = query(mindate, maxdate, minlon, maxlon,
                   minlat, maxlat, mindepth, maxdepth)

In [89]:
response = requests.post(
    f"{base_url}/api/query",
    json.dumps(query_body),
    headers={"Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [90]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/Argo_UCA_Testbed_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc",
    "wb",
).write(response.content)
df = xr.open_dataset(
    f"./Beacon_V1.0.0_Output/Argo_UCA_Testbed_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc"
).to_dataframe()
df

,TEMP,TEMP_QC,TEMP_units,TEMP_standard_name,PSAL,PSAL_standard_name,TIME,DEPTH,LONGITUDE,LATITUDE,DATASET
obs,,,,,,,,,,,
0,-0.444000,1,degree_Celsius,sea_water_temperature,34.223999,sea_water_salinity,2018-12-16 16:58:00,8.8,-0.005762,-66.199252,coriolis/2903715/2903715_prof.nc
1,-0.692000,1,degree_Celsius,sea_water_temperature,34.207001,sea_water_salinity,2018-12-18 04:44:19,5.1,-0.068284,-66.220430,coriolis/2903715/2903715_prof.nc
2,-0.906000,1,degree_Celsius,sea_water_temperature,34.208000,sea_water_salinity,2018-12-18 04:44:19,6.1,-0.068284,-66.220430,coriolis/2903715/2903715_prof.nc
3,-0.839000,1,degree_Celsius,sea_water_temperature,34.215000,sea_water_salinity,2018-12-18 04:44:19,7.1,-0.068284,-66.220430,coriolis/2903715/2903715_prof.nc
4,-0.811000,1,degree_Celsius,sea_water_temperature,34.226002,sea_water_salinity,2018-12-18 04:44:19,8.1,-0.068284,-66.220430,coriolis/2903715/2903715_prof.nc
...,...,...,...,...,...,...,...,...,...,...,...
981406,27.686001,1,degree_Celsius,sea_water_temperature,34.495998,sea_water_salinity,2018-11-21 13:11:09,4.6,-163.243000,15.810000,aoml/4901443/4901443_prof.nc
981407,27.597000,1,degree_Celsius,sea_water_temperature,34.426998,sea_water_salinity,2018-12-02 02:13:00,4.6,-163.434000,16.067000,aoml/4901443/4901443_prof.nc
981408,27.523001,1,degree_Celsius,sea_water_temperature,34.436001,sea_water_salinity,2018-12-02 02:13:00,9.9,-163.434000,16.067000,aoml/4901443/4901443_prof.nc
